In [38]:
import pandas as pd
from RT_prediction.RT_prediction import *
import numpy as np
from copy import deepcopy

In [2]:
datadir = "../MSMS/"
pols = ["neg/","pos/"]
inHouseResultFile = "combined_resultsTop1_above80_in-house.csv_decoID.csv"
onlineResultFile = "combined_resultsTop1_above80.csv_decoID.csv"


In [7]:
mergedResults = {}
for pol in pols:
    tmp = pd.read_csv(datadir + pol + inHouseResultFile,index_col=0)
    onlineResults = pd.read_csv(datadir + pol + onlineResultFile,index_col=0)
    toDrop = [x for x in onlineResults.index.values if x in tmp.index.values]
    onlineResults = onlineResults.drop(toDrop)
    mergedResults[pol] = pd.concat((tmp,onlineResults),axis=0,ignore_index=False)
    print(len(mergedResults[pol]))
mergedResults[pol]

256
325


,isolation_center_m/z,rt,compound_m/z,compound_rt,compound_formula,DB_Compound_ID,Compound_Name,DB_Spectrum_ID,dot_product,ppm_Error,isotope_similiarity,Abundance,ComponentID,redundant,Origin File
#featureID,,,,,,,,,,,,,,,
47,90.054848,6.146067,90.055000,6.13,C3H7NO2,56-41-7,L-ALANINE,20,99.956839,1.691000,0.500000,1.000000,original,False,../MSMS/pos/data/QTOF/BatchU_pP_BatchQPool_Ite...
108,114.066287,2.965833,114.066200,2.77,C4H7N3O,60-27-5,CREATININE,651,99.982898,-0.766654,0.500000,1.000000,original,False,../MSMS/pos/data/QTOF/BatchL_pP_Pool_Iterative...
120,116.070714,5.001767,116.070600,4.97,C5H9NO2,147-85-3,PROLINE,466,99.994116,-0.984872,0.500000,1.000000,original,False,../MSMS/pos/data/QTOF/BatchU_pP_BatchQPool_Ite...
126,118.086334,4.909100,118.086300,4.71,C5H11NO2,760-78-1,NORVALINE,2393,99.998212,-0.289860,0.500000,0.897079,760-78-1,True,../MSMS/pos/data/QTOF/BatchL_pP_Pool_Iterative...
134,120.065443,5.890767,120.065500,6.33,C4H9NO3,672-15-1,HOMOSERINE,1056,99.973439,0.474417,0.500000,1.000000,original,False,../MSMS/pos/data/QTOF/BatchU_pP_BatchQPool_Ite...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5479,199.107723,9.935338,199.108000,-1.00,C9H14N2O3,UBLWFFBGMBRBMC-UHFFFAOYSA-N,NCGC00380778-01_C9H14N2O3_(3S_8aS)-3-[(1R)-1-H...,2755253,82.779381,1.390021,0.983244,0.940493,UBLWFFBGMBRBMC-UHFFFAOYSA-N,True,../MSMS/pos/data/ORBI/pP_BatchG_Pool_ID_02
5481,359.315125,0.889900,359.315500,-1.00,C21H42O4,VBICKXHEKHSIBG-UHFFFAOYSA-N,1-Monostearin,1580497,96.750839,1.045011,0.500000,0.000000,original,False,../MSMS/pos/data/QTOF/BatchL_pP_Pool_Iterative...
5489,489.355408,0.689312,489.357400,-1.00,C30H48O5,JPGOJQJBPLCRQP-YHJYJCICSA-N,(1R_2R_4aS_6aS_6bR_10S_12aR_14bS)-1_8_10-trihy...,2781090,94.219342,4.071244,0.500000,0.000000,original,False,../MSMS/pos/data/ORBI/pP_BatchF_Pool_ID_04


In [9]:
rtsToPredict = []
for pol in mergedResults:
    for index,row in mergedResults[pol].iterrows():
        if row["compound_rt"] < 0:
            rtsToPredict.append(row["DB_Compound_ID"])
rtsToPredict = list(set(rtsToPredict))
print(len(rtsToPredict))

386


In [11]:
smilesMapping  = {}
for cpd in rtsToPredict:
    tmp = InChiKeyToSMILES(cpd)
    if len(tmp) > 0:
        smilesMapping[cpd] = tmp
print(len(smilesMapping))

319


In [16]:
RT_pred = RT_predictor(n_jobs=4)
tmp = pd.DataFrame.from_dict(smilesMapping,orient="index").rename(mapper={0:"SMILES"},axis=1)
props = RT_pred.computeMolecularDescriptors(tmp)

In [19]:
training_data = RT_pred.get_training_data_pHILIC_IROA()
goodCols_train = RT_pred.get_good_desc(training_data)
goodCols_pred = RT_pred.get_good_desc(props)
preds = list(set(goodCols_train).intersection(set(goodCols_pred)))
nBest,mBest,max_depth = (100,51,200)


Unnamed: 0 : non-real valued descriptor
Name : non-real valued descriptor
Unnamed: 0 : non-real valued descriptor
data[i, 1] : non-real valued descriptor


In [20]:
print(RT_pred.crossValPrediction(training_data[preds].to_numpy(),training_data["RT"].values,nBest,mBest,max_depth))
RT_pred.fit(training_data[preds].to_numpy(),training_data["RT"].values,nBest,mBest,max_depth)

0.6591688642767775


In [21]:
#predict RTs
rt_predicted = RT_pred.predict(props[preds].to_numpy())

In [23]:
names = props[props.columns.values[-1]].values
result_rt = pd.DataFrame.from_dict({n:{"name":n,"RT":rt} for n,rt in zip(names,rt_predicted)},orient="index")
result_rt

,name,RT
KIHBGTRZFAVZRV-QGZVFWFLSA-N,KIHBGTRZFAVZRV-QGZVFWFLSA-N,1.899455
BMVXCPBXGZKUPN-UHFFFAOYSA-N,BMVXCPBXGZKUPN-UHFFFAOYSA-N,6.162650
DZKIUEHLEXLYKM-UHFFFAOYSA-N,DZKIUEHLEXLYKM-UHFFFAOYSA-N,2.511135
ZLLQQKITWRWKTD-UHFFFAOYSA-N,ZLLQQKITWRWKTD-UHFFFAOYSA-N,1.960285
HXEACLLIILLPRG-UHFFFAOYSA-N,HXEACLLIILLPRG-UHFFFAOYSA-N,4.546190
...,...,...
OEYIOHPDSNJKLS-UHFFFAOYSA-N,OEYIOHPDSNJKLS-UHFFFAOYSA-N,6.357407
FEWJPZIEWOKRBE-UHFFFAOYSA-N,FEWJPZIEWOKRBE-UHFFFAOYSA-N,7.655167
JZUFKLXOESDKRF-UHFFFAOYSA-N,JZUFKLXOESDKRF-UHFFFAOYSA-N,3.006823
KVXLWOCKZPMXJX-UYAOXDASSA-N,KVXLWOCKZPMXJX-UYAOXDASSA-N,1.524265


In [24]:
for pol in mergedResults:
    for index,row in mergedResults[pol].iterrows():
        if row["compound_rt"] < 0:
            if row["DB_Compound_ID"] in result_rt.index.values:
                mergedResults[pol].at[index,"compound_rt"] = result_rt.at[row["DB_Compound_ID"],"RT"]
mergedResults[pol]

,isolation_center_m/z,rt,compound_m/z,compound_rt,compound_formula,DB_Compound_ID,Compound_Name,DB_Spectrum_ID,dot_product,ppm_Error,isotope_similiarity,Abundance,ComponentID,redundant,Origin File
#featureID,,,,,,,,,,,,,,,
47,90.054848,6.146067,90.055000,6.130000,C3H7NO2,56-41-7,L-ALANINE,20,99.956839,1.691000,0.500000,1.000000,original,False,../MSMS/pos/data/QTOF/BatchU_pP_BatchQPool_Ite...
108,114.066287,2.965833,114.066200,2.770000,C4H7N3O,60-27-5,CREATININE,651,99.982898,-0.766654,0.500000,1.000000,original,False,../MSMS/pos/data/QTOF/BatchL_pP_Pool_Iterative...
120,116.070714,5.001767,116.070600,4.970000,C5H9NO2,147-85-3,PROLINE,466,99.994116,-0.984872,0.500000,1.000000,original,False,../MSMS/pos/data/QTOF/BatchU_pP_BatchQPool_Ite...
126,118.086334,4.909100,118.086300,4.710000,C5H11NO2,760-78-1,NORVALINE,2393,99.998212,-0.289860,0.500000,0.897079,760-78-1,True,../MSMS/pos/data/QTOF/BatchL_pP_Pool_Iterative...
134,120.065443,5.890767,120.065500,6.330000,C4H9NO3,672-15-1,HOMOSERINE,1056,99.973439,0.474417,0.500000,1.000000,original,False,../MSMS/pos/data/QTOF/BatchU_pP_BatchQPool_Ite...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5479,199.107723,9.935338,199.108000,-1.000000,C9H14N2O3,UBLWFFBGMBRBMC-UHFFFAOYSA-N,NCGC00380778-01_C9H14N2O3_(3S_8aS)-3-[(1R)-1-H...,2755253,82.779381,1.390021,0.983244,0.940493,UBLWFFBGMBRBMC-UHFFFAOYSA-N,True,../MSMS/pos/data/ORBI/pP_BatchG_Pool_ID_02
5481,359.315125,0.889900,359.315500,1.921930,C21H42O4,VBICKXHEKHSIBG-UHFFFAOYSA-N,1-Monostearin,1580497,96.750839,1.045011,0.500000,0.000000,original,False,../MSMS/pos/data/QTOF/BatchL_pP_Pool_Iterative...
5489,489.355408,0.689312,489.357400,-1.000000,C30H48O5,JPGOJQJBPLCRQP-YHJYJCICSA-N,(1R_2R_4aS_6aS_6bR_10S_12aR_14bS)-1_8_10-trihy...,2781090,94.219342,4.071244,0.500000,0.000000,original,False,../MSMS/pos/data/ORBI/pP_BatchF_Pool_ID_04


In [25]:
rt_cutoff = 2.0
for pol in mergedResults:
    toDrop = []
    for index,row in mergedResults[pol].iterrows():
        if np.abs(row["rt"] - row["compound_rt"]) > rt_cutoff or row["compound_rt"] < 0:
            toDrop.append(index)
    mergedResults[pol] = mergedResults[pol].drop(toDrop)
mergedResults[pol]

,isolation_center_m/z,rt,compound_m/z,compound_rt,compound_formula,DB_Compound_ID,Compound_Name,DB_Spectrum_ID,dot_product,ppm_Error,isotope_similiarity,Abundance,ComponentID,redundant,Origin File
#featureID,,,,,,,,,,,,,,,
47,90.054848,6.146067,90.055000,6.130000,C3H7NO2,56-41-7,L-ALANINE,20,99.956839,1.691000,0.5,1.000000,original,False,../MSMS/pos/data/QTOF/BatchU_pP_BatchQPool_Ite...
108,114.066287,2.965833,114.066200,2.770000,C4H7N3O,60-27-5,CREATININE,651,99.982898,-0.766654,0.5,1.000000,original,False,../MSMS/pos/data/QTOF/BatchL_pP_Pool_Iterative...
120,116.070714,5.001767,116.070600,4.970000,C5H9NO2,147-85-3,PROLINE,466,99.994116,-0.984872,0.5,1.000000,original,False,../MSMS/pos/data/QTOF/BatchU_pP_BatchQPool_Ite...
126,118.086334,4.909100,118.086300,4.710000,C5H11NO2,760-78-1,NORVALINE,2393,99.998212,-0.289860,0.5,0.897079,760-78-1,True,../MSMS/pos/data/QTOF/BatchL_pP_Pool_Iterative...
134,120.065443,5.890767,120.065500,6.330000,C4H9NO3,672-15-1,HOMOSERINE,1056,99.973439,0.474417,0.5,1.000000,original,False,../MSMS/pos/data/QTOF/BatchU_pP_BatchQPool_Ite...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5418,164.143585,0.954933,164.143400,2.665375,C11H17N,DGFTWBUZRHAHTH-UHFFFAOYSA-N,4 Pentylaniline,595940,99.677981,-1.131039,0.5,0.000000,original,False,../MSMS/pos/data/QTOF/BatchR_pP_BatchNPool_Bla...
5426,423.170074,0.779533,423.169500,2.740180,C22H23ClN6O,PSIFNNKUMBGKDQ-UHFFFAOYSA-N,Losartan,FiehnHILIC000495,98.254449,-1.357522,0.5,0.000000,original,False,../MSMS/pos/data/QTOF/BatchT_pP_BatchJPool_Ite...
5469,166.122833,1.008204,166.122600,2.629415,C10H12O,ZFMSMUAANRJZFM-UHFFFAOYSA-N,Estragole,VF-NPL-QEHF003214,92.683367,-1.404093,0.5,0.000000,original,False,../MSMS/pos/data/ORBI/pP_BatchS_Pool_ID_02


In [27]:
for pol in mergedResults:
    dups = []
    for index,row in mergedResults[pol].iterrows():
        if len([x for x in mergedResults[pol]["DB_Compound_ID"] if x == row["DB_Compound_ID"]]) > 1:
            dups.append("yes")
        else:
            dups.append("no")
    mergedResults[pol]["Compound Matched to Multiple Feats"] = dups
mergedResults[pol]  

,isolation_center_m/z,rt,compound_m/z,compound_rt,compound_formula,DB_Compound_ID,Compound_Name,DB_Spectrum_ID,dot_product,ppm_Error,isotope_similiarity,Abundance,ComponentID,redundant,Origin File,Compound Matched to Multiple Feats
#featureID,,,,,,,,,,,,,,,,
47,90.054848,6.146067,90.055000,6.130000,C3H7NO2,56-41-7,L-ALANINE,20,99.956839,1.691000,0.5,1.000000,original,False,../MSMS/pos/data/QTOF/BatchU_pP_BatchQPool_Ite...,no
108,114.066287,2.965833,114.066200,2.770000,C4H7N3O,60-27-5,CREATININE,651,99.982898,-0.766654,0.5,1.000000,original,False,../MSMS/pos/data/QTOF/BatchL_pP_Pool_Iterative...,no
120,116.070714,5.001767,116.070600,4.970000,C5H9NO2,147-85-3,PROLINE,466,99.994116,-0.984872,0.5,1.000000,original,False,../MSMS/pos/data/QTOF/BatchU_pP_BatchQPool_Ite...,no
126,118.086334,4.909100,118.086300,4.710000,C5H11NO2,760-78-1,NORVALINE,2393,99.998212,-0.289860,0.5,0.897079,760-78-1,True,../MSMS/pos/data/QTOF/BatchL_pP_Pool_Iterative...,no
134,120.065443,5.890767,120.065500,6.330000,C4H9NO3,672-15-1,HOMOSERINE,1056,99.973439,0.474417,0.5,1.000000,original,False,../MSMS/pos/data/QTOF/BatchU_pP_BatchQPool_Ite...,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5418,164.143585,0.954933,164.143400,2.665375,C11H17N,DGFTWBUZRHAHTH-UHFFFAOYSA-N,4 Pentylaniline,595940,99.677981,-1.131039,0.5,0.000000,original,False,../MSMS/pos/data/QTOF/BatchR_pP_BatchNPool_Bla...,no
5426,423.170074,0.779533,423.169500,2.740180,C22H23ClN6O,PSIFNNKUMBGKDQ-UHFFFAOYSA-N,Losartan,FiehnHILIC000495,98.254449,-1.357522,0.5,0.000000,original,False,../MSMS/pos/data/QTOF/BatchT_pP_BatchJPool_Ite...,no
5469,166.122833,1.008204,166.122600,2.629415,C10H12O,ZFMSMUAANRJZFM-UHFFFAOYSA-N,Estragole,VF-NPL-QEHF003214,92.683367,-1.404093,0.5,0.000000,original,False,../MSMS/pos/data/ORBI/pP_BatchS_Pool_ID_02,no


In [35]:
ppmTol = 20
rtTol = .5
pols = ["pos/","neg/"]

foundInOther = []
matchedInOther = []
for index,row in mergedResults[pols[0]].iterrows():
    adjustedMass = row["isolation_center_m/z"] - 2*1.0073
    delta = ppmTol * adjustedMass / 1e6
    bounds = [adjustedMass-delta, adjustedMass + delta]
    unique = True
    for index2,row2 in mergedResults[pols[1]].iterrows():
        if row2["isolation_center_m/z"] > bounds[0] and row2["isolation_center_m/z"] < bounds[1]:
            if np.abs(row2["rt"] - row["rt"]) < rtTol:
                foundInOther.append(index2)
                if row["DB_Compound_ID"] == row2["DB_Compound_ID"]:
                    matchedInOther.append("yes")
                else:
                    matchedInOther.append("no")
                unique = False
                break
    if unique:
        matchedInOther.append("no")
        foundInOther.append("no")

mergedResults[pols[0]]["found in other polarity"] = foundInOther
mergedResults[pols[0]]["same ID in other polarity"] = matchedInOther

    
foundInOther = []
matchedInOther = []
for index,row in mergedResults[pols[1]].iterrows():
    adjustedMass = row["isolation_center_m/z"] - 2*1.0073
    delta = ppmTol * adjustedMass / 1e6
    bounds = [adjustedMass-delta, adjustedMass + delta]
    unique = True
    for index2,row2 in mergedResults[pols[0]].iterrows():
        if row2["isolation_center_m/z"] > bounds[0] and row2["isolation_center_m/z"] < bounds[1]:
            if np.abs(row2["rt"] - row["rt"]) < rtTol:
                foundInOther.append(index2)
                if row["DB_Compound_ID"] == row2["DB_Compound_ID"]:
                    matchedInOther.append("yes")
                else:
                    matchedInOther.append("no")
                unique = False
                break
    if unique:
        matchedInOther.append("no")
        foundInOther.append("no")
    
mergedResults[pols[1]]["found in other polarity"] = foundInOther
mergedResults[pols[1]]["same ID in other polarity"] = matchedInOther
                

In [37]:
for pol in mergedResults:
    mergedResults[pol].to_csv(datadir + pol + "filtered_results_combined.csv")
    print(len(mergedResults[pol]))

159
164


In [41]:
transitionList = {}
polMapper = {"pos/":1,"neg/":-1}
for pol in mergedResults:
    tmp = deepcopy(mergedResults[pol])
    tmp["Molecule List Name"] = ["LLFS Polars " + pol[:-1] for _ in range(len(tmp))]
    tmp["Precursor Charge"] = [polMapper[pol] for _ in range(len(tmp))]
    tmp = tmp.rename({"Compound_Name":"Precursor Name","isolation_center_m/z":"Precursor m/z","rt":"Explicit Retention Time"},axis=1)
    tmp = tmp[["Molecule List Name","Precursor Name","Precursor m/z","Explicit Retention Time","Precursor Charge"]]
    transitionList[pol] = tmp
    

In [44]:
for pol in transitionList:
    transitionList[pol].to_csv(datadir + pol + "putative_IDS_transition_list.csv",index=False)
